# A Year of Pumpkin Prices
*Pumpkin Prices in 13 US Cities: 2016-2017*

- Which states produce the most pumpkin?
- Where are pumpkin prices highest?
- How does pumpkin size relate to price?
- Which pumpkin variety is the most expensive? Least expensive?

Data for each city includes the following columns (although not all information is available for every city)
```
Commodity Name: Always pumpkin, since this is a pumpkin-only dataset
City Name: City where the pumpkin was sold
Type
Package
Variety
Sub Variety
Grade: In the US, usually only canned pumpkin is graded
Date: Date of sale (rounded up to the nearest Saturday)
Low Price
High Price
Mostly Low
Mostly High
Origin: Where the pumpkins were grown
Origin District
Item Size
Color
Environment
Unit of Sale
Quality
Condition
Appearance
Storage
Crop
Repack: Whether the pumpkin has been repackaged before sale
Trans Mode
```



# Loading Data

In [0]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import plotly.express as px

In [0]:
path = '/content/drive/My Drive/courses/data analytics/datasets/pumpkinprices'
filenames,totalfiles = os.listdir(path), len(os.listdir(path))

In [28]:
filenames

['atlanta_9-24-2016_9-30-2017.csv',
 'columbia_9-24-2016_9-30-2017.csv',
 'boston_9-24-2016_9-30-2017.csv',
 'dallas_9-24-2016_9-30-2017.csv',
 'baltimore_9-24-2016_9-30-2017.csv',
 'chicago_9-24-2016_9-30-2017.csv',
 'los-angeles_9-24-2016_9-30-2017.csv',
 'new-york_9-24-2016_9-30-2017.csv',
 'detroit_9-24-2016_9-30-2017.csv',
 'miami_9-24-2016_9-30-2017.csv',
 'san-fransisco_9-24-2016_9-30-2017.csv',
 'philadelphia_9-24-2016_9-30-2017.csv',
 'st-louis_9-24-2016_9-30-2017.csv']

In [5]:
print('total file',totalfiles)

total file 13


In [0]:
df0 = pd.read_csv(f'/content/drive/My Drive/courses/data analytics/datasets/pumpkinprices/{filenames[0]}')
df1 = pd.read_csv(f'/content/drive/My Drive/courses/data analytics/datasets/pumpkinprices/{filenames[1]}')

In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 25 columns):
Commodity Name     263 non-null object
City Name          263 non-null object
Type               0 non-null float64
Package            263 non-null object
Variety            263 non-null object
Sub Variety        54 non-null object
Grade              0 non-null float64
Date               263 non-null object
Low Price          263 non-null int64
High Price         263 non-null int64
Mostly Low         263 non-null int64
Mostly High        263 non-null int64
Origin             263 non-null object
Origin District    0 non-null float64
Item Size          238 non-null object
Color              161 non-null object
Environment        0 non-null float64
Unit of Sale       0 non-null float64
Quality            0 non-null float64
Condition          0 non-null float64
Appearance         0 non-null float64
Storage            0 non-null float64
Crop               0 non-null float64
Repack       

In [21]:
df1.head()

,Commodity Name,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,Mostly Low,Mostly High,Origin,Origin District,Item Size,Color,Environment,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode
0,PUMPKINS,COLUMBIA,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,10/15/2016,120,120,120,120,NORTH CAROLINA,NaN,med,ORANGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,PUMPKINS,COLUMBIA,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,10/22/2016,120,120,120,120,NORTH CAROLINA,NaN,med,ORANGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,PUMPKINS,COLUMBIA,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,10/29/2016,120,120,120,120,NORTH CAROLINA,NaN,med,ORANGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,PUMPKINS,COLUMBIA,NaN,36 inch bins,HOWDEN TYPE,NaN,NaN,09/24/2016,150,150,150,150,MICHIGAN,NaN,lge,ORANGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,PUMPKINS,COLUMBIA,NaN,36 inch bins,HOWDEN TYPE,NaN,NaN,09/24/2016,160,160,160,160,NORTH CAROLINA,NaN,jbo,ORANGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [27]:
df1.describe(include='all')

,Commodity Name,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,Mostly Low,Mostly High,Origin,Origin District,Item Size,Color,Environment,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode
count,263,263,0.0,263,263,54,0.0,263,263.000000,263.000000,263.000000,263.000000,263,0.0,238,161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263,0.0
unique,1,1,NaN,5,9,1,NaN,12,NaN,NaN,NaN,NaN,5,NaN,7,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
top,PUMPKINS,COLUMBIA,NaN,24 inch bins,HOWDEN TYPE,FLAT TYPE,NaN,10/29/2016,NaN,NaN,NaN,NaN,PENNSYLVANIA,NaN,med,ORANGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
freq,263,263,NaN,110,91,54,NaN,34,NaN,NaN,NaN,NaN,143,NaN,82,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.372624,141.790875,141.372624,141.790875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.414667,75.496916,75.414667,75.496916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.000000,15.000000,15.000000,15.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.000000,125.000000,125.000000,125.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.000000,150.000000,150.000000,150.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.000000,200.000000,200.000000,200.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
dataset = df1[['Package','Variety','Date','Origin','Item Size','Color','High Price']]

In [34]:
pd.DatetimeIndex('Date')

,Package,Variety,Date,Origin,Item Size,Color,High Price
0,24 inch bins,HOWDEN TYPE,10/15/2016,NORTH CAROLINA,med,ORANGE,120
1,24 inch bins,HOWDEN TYPE,10/22/2016,NORTH CAROLINA,med,ORANGE,120
2,24 inch bins,HOWDEN TYPE,10/29/2016,NORTH CAROLINA,med,ORANGE,120
3,36 inch bins,HOWDEN TYPE,09/24/2016,MICHIGAN,lge,ORANGE,150
4,36 inch bins,HOWDEN TYPE,09/24/2016,NORTH CAROLINA,jbo,ORANGE,160
...,...,...,...,...,...,...,...
258,bushel cartons,MINIATURE,10/22/2016,PENNSYLVANIA,sml,ORANGE,28
259,bushel cartons,MINIATURE,10/22/2016,PENNSYLVANIA,sml,WHITE,28
260,bushel cartons,MINIATURE,10/29/2016,PENNSYLVANIA,lge,ORANGE,30
261,bushel cartons,MINIATURE,10/29/2016,PENNSYLVANIA,sml,ORANGE,28


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_features = ['Item Size']
numerical_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_features = ['Package', 'Variety', 'Origin', 'Color']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])

# Split the data into training and test sets
X = dataset.drop('High Price', axis=1)
y = dataset['High Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Preprocessing of training data, fit model 
pipeline.fit(X_train, y_train)

# Preprocessing of test data, get predictions
predictions = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
print('Mean Absolute Error:', mae)